In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [8]:
import pandas as pd
df = pd.read_csv('Final_Dataset.txt', index_col=0)
#df = df[['user_id_maped','duration_seconds','program_class','program_genre','series_title', 'hd','original_name']]

In [10]:
len(df)

3598607

In [11]:
def standardize_text(input_text):
    clean_text = re.sub('[^A-Za-z0-9]+', '', str(input_text))
    return clean_text
df['standardize_original_name'] = df['original_name'].apply(lambda x: standardize_text(x))

In [12]:
#df = df.groupby(['user_id_maped','series_title','standardize_original_name'], as_index=False)['duration_seconds'].sum()
#df = df.groupby(['user_id_maped','season','standardize_original_name'], as_index=False)['duration_seconds'].sum()
#df = df.groupby(['user_id_maped','episode','standardize_original_name'], as_index=False)['duration_seconds'].sum()
df = df.groupby(['user_id_maped','program_class','program_genre','series_title', 'hd','standardize_original_name'], as_index=False)['duration_seconds'].sum()

In [13]:
len(df)

714700

In [15]:
df = df.copy()
#df['duration_seconds_cap'] = np.where(df.duration_seconds>=(60*60*3), (60*60*3), df.duration_seconds)
df['duration_seconds_cap'] = np.where(df.duration_seconds>=(60*60*1), (60*60*1), df.duration_seconds)
df = df.copy()
#IMDB actual durations from others APIs, but because we don't have actual movies IDs we will rely on max observed duration
df['max_duration'] = df.groupby('standardize_original_name')['duration_seconds_cap'].transform(lambda x: x.max())
df['rating'] = df['duration_seconds_cap']*1.0/df['max_duration']

In [16]:
name_to_idx = {mid: idx for idx, mid in enumerate(df['standardize_original_name'].unique())}
df['movie_id'] = df['standardize_original_name'].map(name_to_idx) #if it's matched with the API will add more features

In [36]:
#Adjust Undefined Genre according to IMDb 
df.loc[ df['standardize_original_name'] == 'Dunkirk', 'program_genre'] = 'Action'
df.loc[ df['standardize_original_name'] == 'Friends', 'program_genre'] = 'Comedy'
df.loc[ df['standardize_original_name'] == 'Harry', 'program_genre'] = 'Crime'
df.loc[ df['standardize_original_name'] == 'SehheeWaaree', 'program_genre'] = 'Others'
df.loc[ df['standardize_original_name'] == 'BatmanUnlimitedAnimalInstinctsforViewing', 'program_genre'] = 'Animation'

In [37]:
df.program_genre.unique()

array(['Action', 'Animation', 'Comedy', 'Family', 'Horror', 'Documentary',
       'Drama', 'Biography', 'Thriller', 'Romance', 'Crime', 'Adventure',
       'Sci-Fi', 'Others', 'Wrestling'], dtype=object)

In [ ]:
#df = df[df.rating > 0]
#x = df.loc[df.rating > 0]
#len(x)

In [ ]:
#df.shape[0]/(len(df.user_id_maped.unique())*len(df.movie_id.unique()))

In [17]:
train = df.sample(frac=0.8,random_state=5) 
test = df.drop(train.index)

#thesis analysis 
test = test.loc[test.movie_id.isin(train.movie_id.unique())]
test = test.loc[test.user_id_maped.isin(train.user_id_maped.unique())]
test.reset_index(drop=True, inplace=True)

In [ ]:
#train.to_csv('train.csv', index=False)
#test.to_csv('test.csv', index=False)
#df.to_csv('df.csv', index= False)

In [ ]:
#Binary Test

In [19]:
#binary
R_Binary = pd.pivot_table(train, values='rating', index='user_id_maped', columns='movie_id', aggfunc='count', fill_value=0)

In [20]:
R_Binary

movie_id,0,1,2,3,4,5,6,7,8,9,...,1783,1784,1785,1787,1788,1789,1790,1791,1792,1793
user_id_maped,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34279,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
34280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#len(train.user_id_maped.unique())

28243

In [21]:
model_train = NMF(n_components=10, random_state=5) 

model_train.fit(R_Binary.values)
nmf_features_train = model_train.transform(R_Binary.values)

In [22]:
#multiplication
matrix_mult_train = np.matmul(nmf_features_train, model_train.components_)

In [23]:
matrix_mult_train = pd.DataFrame(data=matrix_mult_train, index = R_Binary.index, columns = R_Binary.columns)
melt_train_predictions = matrix_mult_train.reset_index().melt(id_vars=['user_id_maped'])

In [24]:
#actual_vs_predicted = pd.merge(melt_train_predictions, train[['user_id_maped', 'movie_id']], how='inner', on=['user_id_maped', 'movie_id'])

actual_vs_predicted = pd.merge(melt_train_predictions, test[['user_id_maped', 'movie_id']], how='inner', on=['user_id_maped', 'movie_id'])
actual_vs_predicted['rating'] = 1

In [25]:
MSE = mean_squared_error(actual_vs_predicted.value, actual_vs_predicted.rating)
RMSE_Binary = np.sqrt(MSE)
MAE_Binary = mean_absolute_error(actual_vs_predicted.value, actual_vs_predicted.rating)

In [26]:
RMSE_Binary

0.7910443508862998

In [27]:
MAE_Binary

0.760474482061099

In [ ]:
#rating test
#rating test
#rating test
#rating test
#rating test

In [28]:
R_Rating = pd.pivot_table(train, values='rating', index='user_id_maped', columns='movie_id', aggfunc='mean', fill_value=0)

In [38]:
R_Rating

movie_id,0,1,2,3,4,5,6,7,8,9,...,1783,1784,1785,1787,1788,1789,1790,1791,1792,1793
user_id_maped,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.024167,0.000556,0.016389,0.024167,0.005556,0.000000,0.000000,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.016667,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0225,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
4,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
5,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34279,0.0,0.000000,0.000000,0.000000,0.016944,0.000000,0.000000,0.000000,0.0000,0.005556,...,0.0,0,0,0,0.0,0,0,0,0,0
34280,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0
34281,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0,0,0,0.0,0,0,0,0,0


In [29]:
#when the component increase it gives higher errors and model startes overfitting
model_train = NMF(n_components=10, random_state=5) 

model_train.fit(R_Rating.values)
nmf_features_train = model_train.transform(R_Rating.values)
matrix_mult_train = np.matmul(nmf_features_train, model_train.components_)

In [30]:
matrix_mult_train = pd.DataFrame(data=matrix_mult_train, index = R_Rating.index, columns = R_Rating.columns)
melt_train_predictions = matrix_mult_train.reset_index().melt(id_vars=['user_id_maped'])

In [31]:
actual_vs_predicted = pd.merge(melt_train_predictions, test[['user_id_maped', 'movie_id', 'rating']], how='inner', on=['user_id_maped', 'movie_id'])

In [32]:
MSE = mean_squared_error(actual_vs_predicted.value, actual_vs_predicted.rating)
RMSE_Rating = np.sqrt(MSE)
MAE_Rating = mean_absolute_error(actual_vs_predicted.value, actual_vs_predicted.rating)

In [33]:
RMSE_Rating

0.38761656748351936

In [34]:
MAE_Rating

0.20626470632256003